In [ ]:
#-*- coding: utf-8 -*-
%matplotlib inline
from functools import reduce
import pandas as pd
import numpy as np
import statistics
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import sys
import traceback
import itertools
import csv
import os
import glob
import math
import wave
import time as tm
import scipy.fftpack
from pylab import *
from scipy import stats
from sklearn import datasets
from sklearn.model_selection import train_test_split
# from sklearn import cross_validation

In [ ]:
def load_labeled_data(csv_path, two_class)

from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import collections
from sklearn.preprocessing import Imputer

#データ準備",
print(csv_path)
csv_files = sorted(glob.glob(csv_path))
data_list = []

for f in csv_files:
    data_list.append(pd.read_csv(f, header=None))
distance = pd.concat(data_list)

distance = distance.fillna(0)
X = distance.iloc[:, 0:shape(distance)[1]-1]
Y = distance.iloc[:, shape(distance)[1] - 1].astype(np.int64)

# #ダウンサンプリング前before",
print("before")
count_dict = collections.Counter(Y)
print(count_dict)

if two_class:
    Y[Y<=4] = 1
    Y[Y>=5] = 0
    # # 負例をダウンサンプリング\n",
    rus = RandomUnderSampler(random_state=1)
    # # 学習用データに反映\n",
    X, Y = rus.fit_sample(X, Y)
    #ダウンサンプリング後\n",
    
else:
    X = X[Y != 8]
    Y = Y[Y != 8][0]
    
# #ダウンサンプリング後after",
count_dict = collections.Counter(Y)
print(count_dict)

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
sc.fit(X)
X = sc.transform(X)

In [ ]:
def show_importances(feature_importances, feature_names, save_path):
    #特徴量の重要度
    feature = feature_importances

#     #特徴量の重要度を上から順に出力する
    f = pd.DataFrame({'number': range(0, len(feature)), 'feature': feature[:]})
    f2 = f.sort_values('feature',ascending=False)
    f3 = f2.ix[:, 'number']

    #特徴量の名前
    label = pd.DataFrame(feature_names)

    #特徴量の重要度順（降順）
    indices = feature
    for i, indice in enumerate(indices):
        print(str(i) + ": " +  str(indice))
    
    for i in range(len(feature)):
        print(str(i + 1) + "   " + str(feature_names[indices[i]]) + "   " + str(feature[indices[i]]))
        
    plt.bar(indices, feature[indices], color='red', align='center')
    plt.xlabel("features")
    plt.ylabel("feature importances")
    plt.xlim([-1, len(feature)])
    plt.tight_layout()
#     plt.show()
    plt.savefig(save_path)
#     return indices, feature_names, feature

In [ ]:
def show_forest():
    tree.export_graphviz(clf, out_file=dot_data,feature_names=train_X.columns, max_depth=3)
    graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
    graph.write_pdf("graph.pdf")
    Image(graph.create_png())

In [ ]:
def random_forest(X, Y, scoring, feature_names, save_path):
    from sklearn.svm import SVC
    from sklearn.ensemble import RandomForestClassifier as RFC
    from sklearn.model_selection import StratifiedKFold
    from sklearn.feature_selection import RFECV
    from sklearn import cross_validation
    from sklearn.metrics import roc_curve, auc, accuracy_score
    from sklearn.metrics import confusion_matrix
    
    #------------------------------------------------------------------------------------------
    #2クラスor多クラス
    #n_fold: 交差検定の回数，num_class: クラスの種類の数，scoring: スコアの種類
    if two_class:
        n_fold = 5
        num_class = 2
        scoring = scoring
    else:
        n_fold = 4
        num_class = 7
        scoring = "accuracy" 
    
    print("n_fold = %d" % n_fold)
    print("num_class: %d" % num_class)
    print("scoring: %s" % scoring)
    k_fold = cross_validation.KFold(n=len(X), n_folds=n_fold, shuffle=True, random_state=0)
    
    #------------------------------------------------------------------------------------------
    print("feature names")
    print(feature_names)
    
#     num_trees = [10, 100]
    num_trees = [10]
#     max_dep = [5, 6, 7, 8,  9, 10]
    max_dep = [9]
    
    #------------------------------------------------------------------------------------------
    #特徴量減らしていくpart1
    for a, nt in enumerate(num_trees):
        for b, md in enumerate(max_dep):
            print("num_trees: %1.1f" % nt)
            print("max_dep: %1.1f" % md)
            
            rfc = RFC(random_state=42, n_estimators=nt, max_depth=md)
#             svm = SVC(kernel='linear', random_state=None)

            skf = StratifiedKFold(n_splits=n_fold, random_state=0, shuffle=True)        
            rfecv = RFECV(estimator=rfc, step=1, cv=skf, scoring=scoring)
            
            if two_class:
                for train, test in k_fold:
                    rfecv.fit(X[train], Y[train])
                    y_pred = rfecv.predict(X[test])
                    
            else:
                Y = np.array(Y)
                b = np.zeros([num_class, num_class])
                c_m = np.zeros([num_class, num_class])
                for train, test in k_fold:
                    rfecv.fit(X[train], Y[train])
                    y_pred = rfecv.predict(X[test])
                    b = confusion_matrix(Y[test], y_pred)
#                     print(b)
                    c_m += b
    
                c_m[0] = c_m[0] / 42
                c_m[1] = c_m[1] / 34
                c_m[2] = c_m[2] / 32
                c_m[3] = c_m[3] / 28
                c_m[4] = c_m[4] / 25             
                c_m[5] = c_m[5] / 15             
                c_m[6] = c_m[6] / 27
                print(c_m*100)
                np.savetxt(save_path, c_m, delimiter=',')
            rfecv.fit(X, Y)
            
            print("features: %s" % np.array(feature_names)[rfecv.support_])
            print("importance_: %s" % rfecv.estimator_.feature_importances_)
            print("Best Score: %f" % rfecv.grid_scores_[np.argmax(rfecv.grid_scores_)])
            print("Optimal number of features : %d" % rfecv.n_features_)
            
#             Plot number of features VS. cross-validation scores
#             plt.figure()
#             plt.xlabel("Number of features selected")
#             plt.ylabel("Cross validation score (nb of correct classifications)")
#             plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
#             plt.show()
            print("-----------------------------")
            
            #特徴量の重要度の算出
#             feature_importances = rfecv.estimator_.feature_importances_   
#             show_importances(feature_importances, feature_names)

In [ ]:
import pickle
import warnings
from sklearn.tree import DecisionTreeClassifier


# scoring = "accuracy"
# scoring = "precision"
# scoring = "recall"
scoring = "f1"

two_class = True
multi_class = not two_class

for i in range(1,11):
    csv_path = any_csv_path_you_want
    feature_names=list(np.arange(0, len(pd.DataFrame(X).columns), 1))
    random_forest(X,Y,scoring, feature_names)